In [2]:
import numpy as np
from inspiral_range.inspiral_range import sensemon_range
from gwpy.timeseries import TimeSeries
from tqdm import tqdm

from aframe_plots.config import DataPaths


def get_psd_for_ifo(fname, ifo, fftlength=8):
    ts = TimeSeries.read(fname, ifo)
    psd = ts.psd(fftlength=fftlength, window="hann", method="median")
    return psd.value, psd.frequencies.value


def compute_range_for_run(run_dir, m1, m2, fftlength=8):
    psds = {i: [] for i in ["H1", "L1"]}
    data_dir = run_dir / "test" / "background"
    if not data_dir.exists():
        return {}

    fnames = list(data_dir.iterdir())
    for fname in tqdm(fnames, desc=run_dir.name):
        for ifo, value in psds.items():
            psd, freqs = get_psd_for_ifo(fname, ifo, fftlength)
            value.append(psd)

    ranges = {}
    for key, value in psds.items():
        value = np.stack(value)
        psd = value.mean(0)
        ranges[key] = sensemon_range(
            freqs,
            psd,
            m1,
            m2,
            horizon=False,
            integrate=True,
            detection_snr=8
        )
    return ranges

In [3]:
ranges = {}
for dirname in DataPaths.INTERVAL_PLOTS.iterdir():
    if not dirname.is_dir():
        continue

    run_range = compute_range_for_run(dirname, 35, 35)
    if run_range:
        ranges[dirname.name] = run_range

FileNotFoundError: [Errno 2] No such file or directory: '/home/ethan.marx/aframe/interval-runs'